In [ ]:
!pip install groq -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
import re
import os
from groq import Groq
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

# Custom_LLM class

In [ ]:
class Custom_LLM:

  def __init__(self, model_name="qwen-2.5-32b"):
    self.model_name = model_name
    self.groq_client = Groq()

  def get_patient_from_db(self, patient_id: str) -> dict:
      # Example mock data matching the data structure
      # This function should be changed to pull data from database.
      return {
          "demographics": {
              "name": "Jane A. Doe",
              "age": 38,
              "sex_assigned_at_birth": "Female"
          },
          "medications": [
                  {
                      "name": "Metformin",
                      "dose": "500 mg",
                      "frequency": "BID",
                      "start_date": "03/01/2023"
                  },
                  {
                      "name": "Lisinopril",
                      "dose": "10 mg",
                      "frequency": "Daily",
                      "start_date": "03/01/2023"
                  }
              ],
          "diagnoses": [
                  {
                      "description": "Type 2 Diabetes Mellitus",
                      "code": "E11.9",
                      "status": "Active"
                  },
                  {
                      "description": "Hypertension",
                      "code": "I10",
                      "status": "Active"
                  }
              ],
          "allergies": [
              {
                  "name": "Shellfish",
                  "type": "Food",
                  "reaction": "Anaphylaxis",
                  "severity": "High"
              },
              {
                  "name": "Peanut",
                  "type": "Food",
                  "reaction": "Anaphylaxis",
                  "severity": "High"
              }
          ],
          "laboratory_results": [
              {
                  "test": "Hemoglobin A1c",
                  "result": 7.2,
                  "units": "%",
                  "date": "03/05/2023"
              },
              {
                  "test": "LDL Cholesterol",
                  "result": 145,
                  "units": "mg/dL",
                  "date": "03/05/2023"
              }
          ]
      }

  def call_groq(self, prompt):
    client = self.groq_client
    completion = client.chat.completions.create(
        model=self.model_name,
        messages=[
            {
                "role": "user",
                "content": prompt if isinstance(prompt, str) else str(prompt)
            }
        ],
        temperature=0.6,
        max_completion_tokens=4096,
        top_p=0.95,
        stream=True,
        stop=None,
    )

    response = ''
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""

    return response

  def parse_output(self, response, return_reasons=False):

    food_pattern = r'\[([^\]]+)\]'
    foods = re.findall(food_pattern, response)

    if return_reasons:
        reason_pattern = r'\(([^)]+)\)'
        reasons = re.findall(reason_pattern, response)

    return (foods, reasons) if return_reasons else foods, []

  def create_medical_prompt(self, patient_id: str, user_query: str) -> str:
    """
    Creates a medical context prompt for LLM using patient EHR data

    Args:
        patient_id: Unique identifier for the patient
        user_query: User's medication-related question

    Returns:
        Formatted prompt string with patient context and query

    Raises:
        ValueError: If patient not found or data is incomplete
    """

    patient_data = self.get_patient_from_db(patient_id)  # Replace with actual DB call

    if not patient_data:
        raise ValueError(f"Patient {patient_id} not found in database")

    try:
        demographics = patient_data.get('demographics', {})
        medications = patient_data.get('medications', {})
        conditions = patient_data.get('diagnoses', {})
        labs = patient_data.get('laboratory_results', [])

        prompt_parts = [
            "You're a helpful medical assistant. Given a patient's medical history you will answer their question."
            "\n=== Demographics ===",
            #f"Patient: {demographics.get('name', 'N/A')}",
            f"Age: {demographics.get('age', 'N/A')}",
            f"Sex: {demographics.get('sex_assigned_at_birth', 'N/A')}",

            "\n== Current Medications ==",
            *[f"- {med['name']} ({med['dose']}, {med['frequency']}) since {med['start_date']}"
              for med in medications],

            "\n== Medical Conditions ==",
            *[f"- {cond['description']} ({cond['code']}) - {cond['status']}"
              for cond in conditions],

            "\n== Recent Lab Results ==",
            *[f"- {lab['test']}: {lab['result']} {lab.get('units', '')} ({lab['date']})"
              for lab in labs[:3]],  # Show last 3 relevant labs

            "\n=== Instructions ===",
            "As a medical assistant, consider:",
            "1. Current medications and potential interactions with patient history",
            "2. Patient's medical conditions and contraindications",
            "3. Recent lab values that might affect safety",
            "4. Provide dosage warnings if suggesting OTC medications",
            "5. Highlight urgent concerns requiring physician consultation",

            "\n=== User Query ===",
            user_query,

            "\n=== Response ===",
            "Answer:"
        ]

        return "\n".join(prompt_parts)

    except KeyError as e:
        raise ValueError(f"Invalid patient data structure: {str(e)}") from e

  def create_dietary_prompt(self, patient_id: str, location: str) -> str:
    """
    Creates a dietary recommendation prompt based on medical history

    Args:
        patient_id: Unique patient identifier
        location: Patient's current location (city/country)

    Returns:
        Formatted prompt for dietary recommendations

    Raises:
        ValueError: For missing patient data or invalid structure
    """
    patient_data = self.get_patient_from_db(patient_id)
    if not patient_data:
        raise ValueError(f"Patient {patient_id} not found")

    try:
        demographics = patient_data.get('demographics', {})
        conditions = patient_data.get('diagnoses', {})
        medications = patient_data.get('medications', {})
        allergies = patient_data.get('allergies', [])
        labs = patient_data.get('laboratory_results', [])

        prompt_parts = [
            "You are a clinical nutritionist. Analyze this patient profile and:",
            "1. List top five foods to avoid with reasons.",
            "2. Consider location-specific food availability.",

            "\n=== Patient Profile ===",
            f"Age: {demographics.get('age', 'N/A')}",
            f"Sex: {demographics.get('sex_assigned_at_birth', 'N/A')}",
            f"Location: {location}",

            "\n== Medical Conditions ==",
            *[f"- {cond['description']} ({cond.get('code', '')})"
              for cond in conditions],

            "\n== Medications ==",
            *[f"- {med['name']} ({med['dose']})" for med in medications],

            "\n== Allergies ==",
            *( [f"- {allergy['name']} ({allergy.get('reaction', '')})" for allergy in allergies]
              if allergies else ["- No known food allergies"] ),

            "\n== Relevant Lab Results ==",
            *[f"- {lab['test']}: {lab['result']} {lab.get('units', '')} (Normal: {lab.get('normal_range', '')})"
              for lab in labs if self.is_relevant_lab(lab['test'])],  # Implement relevance filtering

            "\n=== Instructions ===",
            "Prioritize recommendations based on:",
            "1. Direct contraindications from medical conditions",
            "2. Medication interactions (e.g., MAOIs & aged cheeses)",
            "3. Allergy safety",
            "4. Lab values indicating dietary needs (e.g., high LDL → limit saturated fats)",
            "5. Regional food common in {location}",

            "\n=== Response Format ===",
            "Your response must be in ENGLISH language and only contain the list and nothing else like explainations. Structure your response like this:",
            "-> [Food](Reason)."
        ]

        return "\n".join(prompt_parts)

    except KeyError as e:
        raise ValueError(f"Invalid data structure: {str(e)}") from e


  def is_relevant_lab(self, test_name: str) -> bool:
      """Determine if lab test is relevant for dietary recommendations"""
      relevant_tests = {
          'Hemoglobin A1c', 'LDL Cholesterol',
          'Serum Potassium', 'Renal Function Tests'
      }
      return test_name in relevant_tests


  """
  ***APIs***
  """

  def med_question_answer(self, patient_id: str, user_query: str):

    prompt = self.create_medical_prompt(patient_id, user_query)
    response = self.call_groq(prompt)

    return response

  def get_dietary_recommendations(self, patient_id: str, location: str):

    prompt = self.create_dietary_prompt(patient_id, location)
    response = self.call_groq(prompt)
    final_response = self.parse_output(response, return_reasons=False)

    return final_response

In [ ]:
llm = Custom_LLM()

patiend_id = "123"
user_query = "What is the summary of my current medical report?"

response = llm.med_question_answer(patiend_id, user_query)
print(response)

diet_rec, _ = llm.get_dietary_recommendations(patiend_id, "New York")
print("\nDiet Rec (Foods to avoid): ")
display(diet_rec)

# risk, _ = llm.get_future_risk(patiend_id)
# print("\nFuture Risks:")
# display(risk)

Based on the information provided, here is a summary of your current medical report:

- **Age and Gender**: You are a 38-year-old female.
- **Medical Conditions**: You have active Type 2 Diabetes Mellitus and Hypertension.
- **Current Medications**:
  - Metformin: 500 mg, twice daily, which helps to control your blood sugar levels.
  - Lisinopril: 10 mg, daily, used to help manage your blood pressure.
- **Recent Lab Results**:
  - Hemoglobin A1c: 7.2% (as of 03/05/2023). This indicates that your average blood glucose levels over the past 2-3 months are higher than the ideal range for people without diabetes (which is typically less than 5.7%), but it is within the target range for someone with diabetes (typically less than 7.0%).
  - LDL Cholesterol: 145 mg/dL (as of 03/05/2023). This level is considered borderline high, and your doctor may recommend lifestyle changes or additional medication to lower it.

**Key Points**:
- Your diabetes and blood pressure are being managed with the me

['Shellfish',
 'Peanut',
 'Processed meats',
 'Full-fat dairy products',
 'Sugary drinks']


Future Risks:


["Given your conditions and medications, you're managing your diabetes and hypertension, but long-term risks include cardiovascular issues and kidney problems.",
 "It's important to keep an eye on symptoms like persistent fatigue, changes in vision, or swelling, which could indicate complications.",
 'Continuing regular exercise, maintaining a balanced diet, and monitoring blood pressure and glucose levels closely can help manage risks.',
 'Staying proactive with regular check-ups and open communication with your healthcare provider is key to long-term health.']

In [ ]:
# What is the summary of my current medical report?
# Can I take ibuprofen for my headache?

user_query = input("User Query:\n")

response = llm.med_question_answer(patiend_id, user_query)
print("\n\nAI Response:\n")
print(response)

User Query:
Can I take ibuprofen for my headache?


AI Response:

Yes, you can take ibuprofen for your headache, but it's important to be cautious and follow the recommended dosage. Ibuprofen is generally safe for occasional use, but given your medical history, there are a few considerations:

1. **Renal Function**: Ibuprofen can affect kidney function, which is a concern for individuals with hypertension and diabetes, as both conditions can impact kidney health. Make sure you are well hydrated when taking ibuprofen.

2. **Blood Pressure**: Ibuprofen can cause an increase in blood pressure, which might affect your hypertension management. Monitor your blood pressure while taking ibuprofen and inform your healthcare provider if you notice any significant changes.

3. **Stomach Irritation**: Ibuprofen can irritate the stomach lining. If you have a history of stomach ulcers or gastritis, or if you experience stomach discomfort, be cautious with the dosage and consider taking it with food.